# Data wrangling of the IEA's *Net Zero by 2050 Roadmap* (2021)

This notebook transform the data supporting the
IEA's [Net Zero by 2050](https://www.iea.org/reports/net-zero-by-2050) flagship report
(published May 2021)  
to the IAMC format using the **pyam** package ([read the docs](https://pyam-iamc.readthedocs.io)).

In [1]:
import pandas as pd
import pyam

<IPython.core.display.Javascript object>

In [2]:
index = dict(model="IEA", scenario="Netzero2050")

## Annex A

Annex A has the scenario projections supporting and quantifying the "Net Zero by 2050 Roadmap".

This script uses the files as published on the IEA data page on May 18, 2021.  
Unfortunately, the IEA does not implement clear versioning of the files, so this script may not work as is if the IEA decides to update these files.

In [3]:
file = "data/NZE2021_AnnexA.xlsx"
df_list = []  # this list will hold the individual refactored IamDataFrames

### Global energy supply ("World_Balance")

#### Total Energy Supply

Read the relevant section of the World_Balance sheet and cast to an **IamDataFrame**.

In [4]:
balances_totals = pd.read_excel(file, sheet_name="World_Balance", header=4, usecols="A:F")[0:17]

In [5]:
df_balances_totals = pyam.IamDataFrame(balances_totals, **index, region="World", variable="Unnamed: 0", unit="EJ")

Rename the fuels to IAMC-style variables similar to those used in the IPCC AR6.

In [6]:
variable_mapping = {
    "Total energy supply" : "Primary Energy",
    "Modern gaseous bioenergy": "Bioenergy|Gaseous",
    "Modern liquid bioenergy": "Bioenergy|Liquid",
    "Modern solid bioenergy": "Bioenergy|Solid",
    "Renewables": "Renewables (excluding traditional biomass)",
    "Traditional use of biomass": "Biomass|Traditional",
    "Coal with CCUS": "Coal|CCUS",
    "Unabated coal": "Coal|Unabated",
    "Natural gas with CCUS": "Natural Gas|CCUS" ,
    "Unabated natural gas": "Natural Gas|Unabated",
    "of which non-energy use": "Oil|Non-energy Use"
}

df_balances_totals.rename(variable=variable_mapping, inplace=True)

Prefix "Primary Energy" to variables.

In [7]:
prefix = "Primary Energy"

df_balances_totals.rename(
    variable=dict([(v, f"{prefix}|{v}") for v in df_balances_totals.variable if v != prefix]),
    inplace=True
)

Compute aggregates of timeseries data that are only given at sub-sectoral detail

In [8]:
agg_vars = ["Primary Energy|Natural Gas", "Primary Energy|Coal", "Primary Energy|Bioenergy"]

for v in agg_vars:
    df_balances_totals.aggregate(v, append=True)

Show the timeseries data in the IAMC format.

In [9]:
df_balances_totals.timeseries()

2019  \
model scenario    region variable                                           unit          
IEA   Netzero2050 World  Primary Energy                                     EJ    612.0   
                         Primary Energy|Bioenergy                           EJ     37.0   
                         Primary Energy|Bioenergy|Gaseous                   EJ      2.0   
                         Primary Energy|Bioenergy|Liquid                    EJ      4.0   
                         Primary Energy|Bioenergy|Solid                     EJ     31.0   
                         Primary Energy|Biomass|Traditional                 EJ     25.0   
                         Primary Energy|Coal                                EJ    160.0   
                         Primary Energy|Coal|CCUS                           EJ      0.0   
                         Primary Energy|Coal|Unabated                       EJ    160.0   
                         Primary Energy|Hydro                               EJ     15.0   
                         Primary Energy|Natural Gas                         EJ    139.0   
                         Primary Energy|Natural Gas|CCUS                    EJ      0.0   
                         Primary Energy|Natural Gas|Unabated                EJ    139.0   
                         Primary Energy|Nuclear                             EJ     30.0   
                         Primary Energy|Oil                                 EJ    190.0   
                         Primary Energy|Oil|Non-energy Use                  EJ     28.0   
                         Primary Energy|Other renewables                    EJ      4.0   
                         Primary Energy|Renewables (excluding traditiona... EJ     67.0   
                         Primary Energy|Solar                               EJ      4.0   
                         Primary Energy|Wind                                EJ      5.0   

                                                                                   2020  \
model scenario    region variable                                           unit          
IEA   Netzero2050 World  Primary Energy                                     EJ    587.0   
                         Primary Energy|Bioenergy                           EJ     37.0   
                         Primary Energy|Bioenergy|Gaseous                   EJ      2.0   
                         Primary Energy|Bioenergy|Liquid                    EJ      3.0   
                         Primary Energy|Bioenergy|Solid                     EJ     32.0   
                         Primary Energy|Biomass|Traditional                 EJ     25.0   
                         Primary Energy|Coal                                EJ    154.0   
                         Primary Energy|Coal|CCUS                           EJ      0.0   
                         Primary Energy|Coal|Unabated                       EJ    154.0   
                         Primary Energy|Hydro                               EJ     16.0   
                         Primary Energy|Natural Gas                         EJ    137.0   
                         Primary Energy|Natural Gas|CCUS                    EJ      1.0   
                         Primary Energy|Natural Gas|Unabated                EJ    136.0   
                         Primary Energy|Nuclear                             EJ     29.0   
                         Primary Energy|Oil                                 EJ    173.0   
                         Primary Energy|Oil|Non-energy Use                  EJ     27.0   
                         Primary Energy|Other renewables                    EJ      5.0   
                         Primary Energy|Renewables (excluding traditiona... EJ     69.0   
                         Primary Energy|Solar                               EJ      5.0   
                         Primary Energy|Wind                                EJ      6.0   

                                                                             

Append the cleaned-up data to the list.

In [10]:
df_list.append(df_balances_totals)

#### Electricity and Heat Supply

Read the relevant section of the World_Balance sheet and cast to an **IamDataFrame**.

In [11]:
balances_elec_heat = pd.read_excel(file, sheet_name="World_Balance", header=4, usecols="A:F")[17:32]

In [12]:
df_balances_elec_heat = pyam.IamDataFrame(balances_elec_heat, **index, region="World", variable="Unnamed: 0", unit="EJ")

Rename the fuels to IAMC-style variables similar to those used in the IPCC AR6.

In [13]:
variable_mapping = {
    "Electricity and heat sectors": "Secondary Energy|Electricity and Heat",
    "Coal with CCUS": "Coal|CCUS",
    "Unabated coal": "Coal|Unabated",
    "Natural gas with CCUS": "Natural Gas|CCUS" ,
    "Unabated natural gas": "Natural Gas|Unabated",
}

df_balances_elec_heat.rename(variable=variable_mapping, inplace=True)

Prefix "Secondary Energy|Electricity and Heat" to variables.

In [14]:
prefix = "Secondary Energy|Electricity and Heat"

df_balances_elec_heat.rename(
    variable=dict([(v, f"{prefix}|{v}") for v in df_balances_elec_heat.variable if v != prefix]),
    inplace=True
)

Compute aggregates of timeseries data that are only given at sub-sectoral detail

In [15]:
agg_vars = [f"Secondary Energy|Electricity and Heat|{v}" for v in ["Natural Gas", "Coal"]]

for v in agg_vars:
    df_balances_elec_heat.aggregate(v, append=True)

Append the cleaned-up data to the list.

In [16]:
df_list.append(df_balances_elec_heat)

## Compile individual components of the reformatted data and export as csv

In [17]:
df = pyam.concat(df_list)

In [18]:
df.to_csv("data/iea_netzero.csv")